In [1]:
import spatialdata
import spatialdata_plot
import napari
import napari_spatialdata

import sopa.segmentation
import sopa.io

import matplotlib.pyplot as plt
import math

In [2]:
path_to_zarr = "/Users/jnimoca/Jose_BI/P26_SOPA_seg/20240716_991_segopt_2.zarr"
sdata = spatialdata.read_zarr(path_to_zarr)

In [3]:
sdata

SpatialData object, with associated Zarr store: /Users/jnimoca/Jose_BI/P26_SOPA_seg/20240716_991_segopt_2.zarr
├── Images
│     └── '991_subset': DataTree[cyx] (15, 8000, 8000), (15, 4000, 4000), (15, 2000, 2000), (15, 1000, 1000), (15, 500, 500)
└── Shapes
      ├── 'cp_ft02_cpt-1': GeoDataFrame shape: (13542, 1) (2D shapes)
      ├── 'cp_ft02_cpt-8': GeoDataFrame shape: (1, 1) (2D shapes)
      ├── 'cp_ft04_cpt-3': GeoDataFrame shape: (16222, 1) (2D shapes)
      ├── 'cp_ft04_cpt-6': GeoDataFrame shape: (3937, 1) (2D shapes)
      ├── 'cp_ft06_cpt-3': GeoDataFrame shape: (23044, 1) (2D shapes)
      ├── 'cp_ft06_cpt-6': GeoDataFrame shape: (20768, 1) (2D shapes)
      ├── 'cp_ft08_cpt-1': GeoDataFrame shape: (22901, 1) (2D shapes)
      ├── 'cp_ft08_cpt-8': GeoDataFrame shape: (24987, 1) (2D shapes)
      └── 'sopa_patches': GeoDataFrame shape: (64, 1) (2D shapes)
with coordinate systems:
    ▸ 'pixels', with elements:
        991_subset (Images), cp_ft02_cpt-1 (Shapes), cp_ft02_cpt-

outside of SOPA I create a new image with the projection of channels I desire  

1. Access image from spatial data as numpy array? or as zarr object? 
2. project desired channels
3. add new image to spatial data
4. segment with cellpose

Other ways around:

Use original image instead of accessing from spatialdata

In [8]:
image = sdata.images['991_subset'].to_numpy()

AttributeError: 'DataTree' object has no attribute 'to_numpy'

In [10]:
image = spatialdata.models.Image2DModel.parse(data=sdata.images['991_subset'])

ValueError: Unsupported data type: <class 'datatree.datatree.DataTree'>.

In [14]:
sdata.images['991_subset']['scale0']

DataTree('scale0', parent="None")
    Dimensions:  (c: 15, y: 8000, x: 8000)
    Coordinates:
      * c        (c) <U8 480B '750_bg' '647_bg' '555_bg' ... 'CD68' 'Ki67' 'DAPI_2'
      * y        (y) float64 64kB 0.5 1.5 2.5 3.5 ... 7.998e+03 7.998e+03 8e+03
      * x        (x) float64 64kB 0.5 1.5 2.5 3.5 ... 7.998e+03 7.998e+03 8e+03
    Data variables:
        image    (c, y, x) uint16 2GB dask.array<chunksize=(1, 1024, 1024), meta=np.ndarray>

In [15]:
image = spatialdata.models.Image2DModel.parse(data=sdata.images['991_subset']['scale0'])

ValueError: Unsupported data type: <class 'datatree.datatree.DataTree'>.

In [16]:
type(sdata.images['991_subset']['scale0'])

datatree.datatree.DataTree

In [19]:
sdata.images['991_subset']['scale0'].data_vars

Data variables:
    image    (c, y, x) uint16 2GB dask.array<chunksize=(1, 1024, 1024), meta=np.ndarray>

In [20]:
sdata.images['991_subset']['scale0'].image

<xarray.DataArray 'image' (c: 15, y: 8000, x: 8000)> Size: 2GB
dask.array<from-zarr, shape=(15, 8000, 8000), dtype=uint16, chunksize=(1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates:
  * c        (c) <U8 480B '750_bg' '647_bg' '555_bg' ... 'CD68' 'Ki67' 'DAPI_2'
  * y        (y) float64 64kB 0.5 1.5 2.5 3.5 ... 7.998e+03 7.998e+03 8e+03
  * x        (x) float64 64kB 0.5 1.5 2.5 3.5 ... 7.998e+03 7.998e+03 8e+03
Attributes:
    transform:  {'pixels': Identity }

In [21]:
type(sdata.images['991_subset']['scale0'].image)

xarray.core.dataarray.DataArray

In [22]:
image = spatialdata.models.Image2DModel.parse(data=sdata.images['991_subset']['scale0'].image)

In [23]:
image

<xarray.DataArray 'image' (c: 15, y: 8000, x: 8000)> Size: 2GB
dask.array<from-zarr, shape=(15, 8000, 8000), dtype=uint16, chunksize=(1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates:
  * c        (c) int64 120B 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14
  * y        (y) float64 64kB 0.5 1.5 2.5 3.5 ... 7.998e+03 7.998e+03 8e+03
  * x        (x) float64 64kB 0.5 1.5 2.5 3.5 ... 7.998e+03 7.998e+03 8e+03
Attributes:
    transform:  {'pixels': Identity }

In [24]:
numpy_array = image.values

In [25]:
type(numpy_array)

numpy.ndarray

In [27]:
numpy_array.shape

(15, 8000, 8000)

In [ ]:
# ok we got it as a numpy array
# we got to know which channels should be projected for membrane segmentation

In [28]:
# side quest, can we perform projection directly on xarray dataarray object?
max_projection = image.max(dim='c')

In [29]:
type(max_projection)

xarray.core.dataarray.DataArray

In [30]:
max_projection

<xarray.DataArray 'image' (y: 8000, x: 8000)> Size: 128MB
dask.array<max-aggregate, shape=(8000, 8000), dtype=uint16, chunksize=(1024, 1024), chunktype=numpy.ndarray>
Coordinates:
  * y        (y) float64 64kB 0.5 1.5 2.5 3.5 ... 7.998e+03 7.998e+03 8e+03
  * x        (x) float64 64kB 0.5 1.5 2.5 3.5 ... 7.998e+03 7.998e+03 8e+03

In [33]:
sdata.add_image(name='max_projection', image=max_projection)

RuntimeError: The functions add_image(), add_labels(), add_points() and add_shapes() have been removed in favor of dict-like access to the elements. Please use the following syntax to add an element:

	sdata.images["image_name"] = image
	sdata.labels["labels_name"] = labels
	...

The new syntax does not automatically updates the disk storage, so you need to call sdata.write() when the in-memory object is ready to be saved.
To save only a new specific element to an existing Zarr storage please use the functions write_image(), write_labels(), write_points(), write_shapes() and write_table(). We are going to make these calls more ergonomic in a follow up PR.

In [35]:
sdata

SpatialData object, with associated Zarr store: /Users/jnimoca/Jose_BI/P26_SOPA_seg/20240716_991_segopt_2.zarr
├── Images
│     └── '991_subset': DataTree[cyx] (15, 8000, 8000), (15, 4000, 4000), (15, 2000, 2000), (15, 1000, 1000), (15, 500, 500)
└── Shapes
      ├── 'cp_ft02_cpt-1': GeoDataFrame shape: (13542, 1) (2D shapes)
      ├── 'cp_ft02_cpt-8': GeoDataFrame shape: (1, 1) (2D shapes)
      ├── 'cp_ft04_cpt-3': GeoDataFrame shape: (16222, 1) (2D shapes)
      ├── 'cp_ft04_cpt-6': GeoDataFrame shape: (3937, 1) (2D shapes)
      ├── 'cp_ft06_cpt-3': GeoDataFrame shape: (23044, 1) (2D shapes)
      ├── 'cp_ft06_cpt-6': GeoDataFrame shape: (20768, 1) (2D shapes)
      ├── 'cp_ft08_cpt-1': GeoDataFrame shape: (22901, 1) (2D shapes)
      ├── 'cp_ft08_cpt-8': GeoDataFrame shape: (24987, 1) (2D shapes)
      └── 'sopa_patches': GeoDataFrame shape: (64, 1) (2D shapes)
with coordinate systems:
    ▸ 'pixels', with elements:
        991_subset (Images), cp_ft02_cpt-1 (Shapes), cp_ft02_cpt-

In [36]:
max_projection

<xarray.DataArray 'image' (y: 8000, x: 8000)> Size: 128MB
dask.array<max-aggregate, shape=(8000, 8000), dtype=uint16, chunksize=(1024, 1024), chunktype=numpy.ndarray>
Coordinates:
  * y        (y) float64 64kB 0.5 1.5 2.5 3.5 ... 7.998e+03 7.998e+03 8e+03
  * x        (x) float64 64kB 0.5 1.5 2.5 3.5 ... 7.998e+03 7.998e+03 8e+03

In [37]:
#transform xarray.DataArray to DataTree

AttributeError: module 'spatialdata.models' has no attribute 'DataTreeModel'

In [38]:
max_p = spatialdata.models.Image2DModel.parse(data=max_projection)

ValueError: Wrong `dims`: ('y', 'x'). Expected ('c', 'y', 'x').

In [39]:
# gotta have three dimensions

# Expand the dimensions to reintroduce the 'c' dimension with size 1
max_projection_expanded = max_projection.expand_dims(dim='c', axis=0)

In [40]:
max_projection_expanded

<xarray.DataArray 'image' (c: 1, y: 8000, x: 8000)> Size: 128MB
dask.array<broadcast_to, shape=(1, 8000, 8000), dtype=uint16, chunksize=(1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates:
  * y        (y) float64 64kB 0.5 1.5 2.5 3.5 ... 7.998e+03 7.998e+03 8e+03
  * x        (x) float64 64kB 0.5 1.5 2.5 3.5 ... 7.998e+03 7.998e+03 8e+03
Dimensions without coordinates: c

In [42]:
max_p = spatialdata.models.Image2DModel.parse(data=max_projection_expanded)

In [43]:
max_p

<xarray.DataArray 'image' (c: 1, y: 8000, x: 8000)> Size: 128MB
dask.array<broadcast_to, shape=(1, 8000, 8000), dtype=uint16, chunksize=(1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates:
  * c        (c) int64 8B 0
  * y        (y) float64 64kB 0.5 1.5 2.5 3.5 ... 7.998e+03 7.998e+03 8e+03
  * x        (x) float64 64kB 0.5 1.5 2.5 3.5 ... 7.998e+03 7.998e+03 8e+03
Attributes:
    transform:  {'global': Identity }

In [44]:
sdata.images['max_projection'] = max_p

In [45]:
sdata

SpatialData object, with associated Zarr store: /Users/jnimoca/Jose_BI/P26_SOPA_seg/20240716_991_segopt_2.zarr
├── Images
│     ├── '991_subset': DataTree[cyx] (15, 8000, 8000), (15, 4000, 4000), (15, 2000, 2000), (15, 1000, 1000), (15, 500, 500)
│     └── 'max_projection': DataArray[cyx] (1, 8000, 8000)
└── Shapes
      ├── 'cp_ft02_cpt-1': GeoDataFrame shape: (13542, 1) (2D shapes)
      ├── 'cp_ft02_cpt-8': GeoDataFrame shape: (1, 1) (2D shapes)
      ├── 'cp_ft04_cpt-3': GeoDataFrame shape: (16222, 1) (2D shapes)
      ├── 'cp_ft04_cpt-6': GeoDataFrame shape: (3937, 1) (2D shapes)
      ├── 'cp_ft06_cpt-3': GeoDataFrame shape: (23044, 1) (2D shapes)
      ├── 'cp_ft06_cpt-6': GeoDataFrame shape: (20768, 1) (2D shapes)
      ├── 'cp_ft08_cpt-1': GeoDataFrame shape: (22901, 1) (2D shapes)
      ├── 'cp_ft08_cpt-8': GeoDataFrame shape: (24987, 1) (2D shapes)
      └── 'sopa_patches': GeoDataFrame shape: (64, 1) (2D shapes)
with coordinate systems:
    ▸ 'global', with elements:
      

In [ ]:
napari_spatialdata.Interactive(sdata)

2024-07-17 11:51:47.961 | INFO     | napari_spatialdata._view:_on_layer_update:327 - Updating layer.
2024-07-17 11:51:47.962 | INFO     | napari_spatialdata._view:_on_layer_update:327 - Updating layer.
/opt/homebrew/Caskroom/mambaforge/base/envs/SOPA3/lib/python3.10/site-packages/anndata/_core/aligned_df.py:67: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
2024-07-17 11:51:55.631 | INFO     | napari_spatialdata._view:_on_layer_update:327 - Updating layer.
2024-07-17 11:51:55.632 | INFO     | napari_spatialdata._view:_on_layer_update:327 - Updating layer.
2024-07-17 11:51:56.998 | INFO     | napari_spatialdata._view:_on_layer_update:327 - Updating layer.
2024-07-17 11:51:57.000 | INFO     | napari_spatialdata._view:_on_layer_update:327 - Updating layer.
2024-07-17 11:51:58.448 | INFO     | napari_spatialdata._view:_on_layer_update:327 - Updating layer.
2024-07-17 11:51:58.450 | INFO     | napari_spatial

# max projection works

In [48]:
# save sdata
sdata.write('20240717_spatialdata.zarr')

INFO     The SpatialData object is not self-contained (i.e. it contains some elements that are Dask-backed from    
         locations outside 20240717_spatialdata.zarr). Please see the documentation of `is_self_contained()` to    
         understand the implications of working with SpatialData objects that are not self-contained.              
INFO     The Zarr backing store has been changed from                                                              
         /Users/jnimoca/Jose_BI/P26_SOPA_seg/20240716_991_segopt_2.zarr the new file path:                         
         20240717_spatialdata.zarr                                                                                 


In [49]:
sdata

SpatialData object, with associated Zarr store: /Users/jnimoca/Jose_BI/P26_SOPA_seg/20240717_spatialdata.zarr
├── Images
│     ├── '991_subset': DataTree[cyx] (15, 8000, 8000), (15, 4000, 4000), (15, 2000, 2000), (15, 1000, 1000), (15, 500, 500)
│     └── 'max_projection': DataArray[cyx] (1, 8000, 8000)
└── Shapes
      ├── 'cp_ft02_cpt-1': GeoDataFrame shape: (13542, 1) (2D shapes)
      ├── 'cp_ft02_cpt-8': GeoDataFrame shape: (1, 1) (2D shapes)
      ├── 'cp_ft04_cpt-3': GeoDataFrame shape: (16222, 1) (2D shapes)
      ├── 'cp_ft04_cpt-6': GeoDataFrame shape: (3937, 1) (2D shapes)
      ├── 'cp_ft06_cpt-3': GeoDataFrame shape: (23044, 1) (2D shapes)
      ├── 'cp_ft06_cpt-6': GeoDataFrame shape: (20768, 1) (2D shapes)
      ├── 'cp_ft08_cpt-1': GeoDataFrame shape: (22901, 1) (2D shapes)
      ├── 'cp_ft08_cpt-8': GeoDataFrame shape: (24987, 1) (2D shapes)
      └── 'sopa_patches': GeoDataFrame shape: (64, 1) (2D shapes)
with coordinate systems:
    ▸ 'global', with elements:
       

In [50]:
sdata.is_backed()

True

In [51]:
sdata.is_self_contained()

False

In [63]:
sdata.images['991_subset']

DataTree('None', parent=None)
├── DataTree('scale0')
│       Dimensions:  (c: 15, y: 8000, x: 8000)
│       Coordinates:
│         * c        (c) <U8 480B '750_bg' '647_bg' '555_bg' ... 'CD68' 'Ki67' 'DAPI_2'
│         * y        (y) float64 64kB 0.5 1.5 2.5 3.5 ... 7.998e+03 7.998e+03 8e+03
│         * x        (x) float64 64kB 0.5 1.5 2.5 3.5 ... 7.998e+03 7.998e+03 8e+03
│       Data variables:
│           image    (c, y, x) uint16 2GB dask.array<chunksize=(1, 1024, 1024), meta=np.ndarray>
├── DataTree('scale1')
│       Dimensions:  (c: 15, y: 4000, x: 4000)
│       Coordinates:
│         * c        (c) <U8 480B '750_bg' '647_bg' '555_bg' ... 'CD68' 'Ki67' 'DAPI_2'
│         * y        (y) float64 32kB 1.0 3.0 5.0 7.0 ... 7.995e+03 7.997e+03 7.999e+03
│         * x        (x) float64 32kB 1.0 3.0 5.0 7.0 ... 7.995e+03 7.997e+03 7.999e+03
│       Data variables:
│           image    (c, y, x) uint16 480MB dask.array<chunksize=(1, 1024, 1024), meta=np.ndarray>
├── DataTree('scale2')
│       Dimensions:  (c: 15, y: 2000, x: 2000)
│       Coordinates:
│         * c        (c) <U8 480B '750_bg' '647_bg' '555_bg' ... 'CD68' 'Ki67' 'DAPI_2'
│         * y        (y) float64 16kB 2.0 6.0 10.0 14.0 ... 7.99e+03 7.994e+03 7.998e+03
│         * x        (x) float64 16kB 2.0 6.0 10.0 14.0 ... 7.99e+03 7.994e+03 7.998e+03
│       Data variables:
│           image    (c, y, x) uint16 120MB dask.array<chunksize=(1, 1024, 1024), meta=np.ndarray>
├── DataTree('scale3')
│       Dimensions:  (c: 15, y: 1000, x: 1000)
│       Coordinates:
│         * c        (c) <U8 480B '750_bg' '647_bg' '555_bg' ... 'CD68' 'Ki67' 'DAPI_2'
│         * y        (y) float64 8kB 4.0 12.0 20.0 28.0 ... 7.98e+03 7.988e+03 7.996e+03
│         * x        (x) float64 8kB 4.0 12.0 20.0 28.0 ... 7.98e+03 7.988e+03 7.996e+03
│       Data variables:
│           image    (c, y, x) uint16 30MB dask.array<chunksize=(1, 1000, 1000), meta=np.ndarray>
└── DataTree('scale4')
        Dimensions:  (c: 15, y: 500, x: 500)
        Coordinates:
          * c        (c) <U8 480B '750_bg' '647_bg' '555_bg' ... 'CD68' 'Ki67' 'DAPI_2'
          * y        (y) float64 4kB 8.0 24.0 40.0 56.0 ... 7.96e+03 7.976e+03 7.992e+03
          * x        (x) float64 4kB 8.0 24.0 40.0 56.0 ... 7.96e+03 7.976e+03 7.992e+03
        Data variables:
            image    (c, y, x) uint16 8MB dask.array<chunksize=(1, 500, 500), meta=np.ndarray>

In [74]:
from sopa._sdata import get_spatial_image
channels = list(get_spatial_image(sdata, key='991_subset').c.values)
channels

['750_bg',
 '647_bg',
 '555_bg',
 '488_bg',
 'DAPI_bg',
 'Vimentin',
 'CD3e',
 'panCK',
 'CD8',
 'DAPI_1',
 'COL1A1',
 'CD20',
 'CD68',
 'Ki67',
 'DAPI_2']

In [75]:
membrane_channels = ['Vimentin', 'CD3e', 'panCK', 'CD8', 'COL1A1', 'CD20', 'CD68']

In [76]:
membrane_channel_index = [channels.index(channel) for channel in membrane_channels]

In [80]:
membrane_channel_index

[5, 6, 7, 8, 10, 11, 12]

In [68]:
image

<xarray.DataArray 'image' (c: 15, y: 8000, x: 8000)> Size: 2GB
dask.array<from-zarr, shape=(15, 8000, 8000), dtype=uint16, chunksize=(1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates:
  * c        (c) int64 120B 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14
  * y        (y) float64 64kB 0.5 1.5 2.5 3.5 ... 7.998e+03 7.998e+03 8e+03
  * x        (x) float64 64kB 0.5 1.5 2.5 3.5 ... 7.998e+03 7.998e+03 8e+03
Attributes:
    transform:  {'pixels': Identity }

In [78]:
# Select specific channels: 2, 3, 6, 8 (indexing from 0)
membrane_image = image.sel(c=membrane_channel_index)

In [79]:
membrane_image

<xarray.DataArray 'image' (c: 7, y: 8000, x: 8000)> Size: 896MB
dask.array<getitem, shape=(7, 8000, 8000), dtype=uint16, chunksize=(1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates:
  * c        (c) int64 56B 5 6 7 8 10 11 12
  * y        (y) float64 64kB 0.5 1.5 2.5 3.5 ... 7.998e+03 7.998e+03 8e+03
  * x        (x) float64 64kB 0.5 1.5 2.5 3.5 ... 7.998e+03 7.998e+03 8e+03
Attributes:
    transform:  {'pixels': Identity }

In [101]:
type(sdata)

spatialdata._core.spatialdata.SpatialData

In [85]:
# Perform quantile projection along the 'c' dimension for a specific quantile (e.g., 0.5 for median)
quantile_projection = membrane_image.chunk({'c': -1}).quantile(0.85, dim='c', keep_attrs=True, interpolation='linear')

In [86]:
quantile_projection

<xarray.DataArray 'image' (y: 8000, x: 8000)> Size: 512MB
dask.array<getitem, shape=(8000, 8000), dtype=float64, chunksize=(1024, 1024), chunktype=numpy.ndarray>
Coordinates:
  * y         (y) float64 64kB 0.5 1.5 2.5 3.5 ... 7.998e+03 7.998e+03 8e+03
  * x         (x) float64 64kB 0.5 1.5 2.5 3.5 ... 7.998e+03 7.998e+03 8e+03
    quantile  float64 8B 0.85
Attributes:
    transform:  {'pixels': Identity }

In [87]:
# Expand dimensions to reintroduce the 'c' dimension with size 1, if needed
quantile_projection_expanded = quantile_projection.expand_dims(dim='c', axis=0)

In [88]:
quantile_projection_expanded

<xarray.DataArray 'image' (c: 1, y: 8000, x: 8000)> Size: 512MB
dask.array<broadcast_to, shape=(1, 8000, 8000), dtype=float64, chunksize=(1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates:
  * y         (y) float64 64kB 0.5 1.5 2.5 3.5 ... 7.998e+03 7.998e+03 8e+03
  * x         (x) float64 64kB 0.5 1.5 2.5 3.5 ... 7.998e+03 7.998e+03 8e+03
    quantile  float64 8B 0.85
Dimensions without coordinates: c
Attributes:
    transform:  {'pixels': Identity }

In [99]:
print(quantile_projection_expanded.max().compute().item())
print(quantile_projection_expanded.min().compute().item())
print(quantile_projection_expanded.mean().compute().item())

11235.9
0.0
484.7281130609374


2024-07-17 13:56:01.725 | INFO     | napari_spatialdata._view:_on_layer_update:327 - Updating layer.
2024-07-17 13:56:01.727 | INFO     | napari_spatialdata._view:_on_layer_update:327 - Updating layer.
2024-07-17 13:56:14.612 | INFO     | napari_spatialdata._view:_on_layer_update:327 - Updating layer.
2024-07-17 13:56:14.613 | INFO     | napari_spatialdata._view:_on_layer_update:327 - Updating layer.


In [89]:
sdata.images['membrane_q85_projection'] = spatialdata.models.Image2DModel.parse(data=quantile_projection_expanded)

In [90]:
sdata

SpatialData object, with associated Zarr store: /Users/jnimoca/Jose_BI/P26_SOPA_seg/20240717_spatialdata.zarr
├── Images
│     ├── '991_subset': DataTree[cyx] (15, 8000, 8000), (15, 4000, 4000), (15, 2000, 2000), (15, 1000, 1000), (15, 500, 500)
│     ├── 'max_projection': DataArray[cyx] (1, 8000, 8000)
│     └── 'membrane_q85_projection': DataArray[cyx] (1, 8000, 8000)
└── Shapes
      ├── 'cp_ft02_cpt-1': GeoDataFrame shape: (13542, 1) (2D shapes)
      ├── 'cp_ft02_cpt-8': GeoDataFrame shape: (1, 1) (2D shapes)
      ├── 'cp_ft04_cpt-3': GeoDataFrame shape: (16222, 1) (2D shapes)
      ├── 'cp_ft04_cpt-6': GeoDataFrame shape: (3937, 1) (2D shapes)
      ├── 'cp_ft06_cpt-3': GeoDataFrame shape: (23044, 1) (2D shapes)
      ├── 'cp_ft06_cpt-6': GeoDataFrame shape: (20768, 1) (2D shapes)
      ├── 'cp_ft08_cpt-1': GeoDataFrame shape: (22901, 1) (2D shapes)
      ├── 'cp_ft08_cpt-8': GeoDataFrame shape: (24987, 1) (2D shapes)
      └── 'sopa_patches': GeoDataFrame shape: (64, 1) (2D sha

In [92]:
sdata.write('20240717_spatialdata_mem.zarr')

INFO     The SpatialData object is not self-contained (i.e. it contains some elements that are Dask-backed from    
         locations outside 20240717_spatialdata_mem.zarr). Please see the documentation of `is_self_contained()` to
         understand the implications of working with SpatialData objects that are not self-contained.              
INFO     The Zarr backing store has been changed from 20240717_spatialdata.zarr the new file path:                 
         20240717_spatialdata_mem.zarr                                                                             


In [93]:
napari_spatialdata.Interactive(sdata)

2024-07-17 13:23:38.943 | INFO     | napari_spatialdata._view:_on_layer_update:327 - Updating layer.
2024-07-17 13:23:38.944 | INFO     | napari_spatialdata._view:_on_layer_update:327 - Updating layer.
/opt/homebrew/Caskroom/mambaforge/base/envs/SOPA3/lib/python3.10/site-packages/anndata/_core/aligned_df.py:67: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
2024-07-17 13:23:43.389 | INFO     | napari_spatialdata._view:_on_layer_update:327 - Updating layer.
2024-07-17 13:23:43.389 | INFO     | napari_spatialdata._view:_on_layer_update:327 - Updating layer.
2024-07-17 13:23:46.375 | INFO     | napari_spatialdata._view:_on_layer_update:327 - Updating layer.
2024-07-17 13:23:46.376 | INFO     | napari_spatialdata._view:_on_layer_update:327 - Updating layer.
2024-07-17 13:23:57.423 | INFO     | napari_spatialdata._view:_on_layer_update:327 - Updating layer.
2024-07-17 13:23:57.425 | INFO     | napari_spatial

In [132]:
import xarray
from sopa._sdata import get_spatial_image

def calculate_quantile(sdata, key:str, channels:list, quantile_value:float=0.9, return_ArrayData:bool=False):

    image = sdata.images[key]['scale0'].image
    assert type(image) == xarray.core.dataarray.DataArray, 'Image is not a xarray DataArray'
    
    selected_channels_image = image.sel(c=channels)

    quantile_projection = selected_channels_image.chunk({'c': -1}).quantile(quantile_value, dim='c', keep_attrs=True, interpolation='linear')
    quantile_projection_expanded = quantile_projection.expand_dims(dim='c', axis=0)

    if return_ArrayData:
        return quantile_projection_expanded
    else:
        sdata.images[f'{key}_q{int(quantile_value*100)}_projection'] = spatialdata.models.Image2DModel.parse(data=quantile_projection_expanded)
        return sdata

In [103]:
sdata

SpatialData object, with associated Zarr store: /Users/jnimoca/Jose_BI/P26_SOPA_seg/20240717_spatialdata_mem.zarr
├── Images
│     ├── '991_subset': DataTree[cyx] (15, 8000, 8000), (15, 4000, 4000), (15, 2000, 2000), (15, 1000, 1000), (15, 500, 500)
│     ├── 'max_projection': DataArray[cyx] (1, 8000, 8000)
│     └── 'membrane_q85_projection': DataArray[cyx] (1, 8000, 8000)
└── Shapes
      ├── 'cp_ft02_cpt-1': GeoDataFrame shape: (13542, 1) (2D shapes)
      ├── 'cp_ft02_cpt-8': GeoDataFrame shape: (1, 1) (2D shapes)
      ├── 'cp_ft04_cpt-3': GeoDataFrame shape: (16222, 1) (2D shapes)
      ├── 'cp_ft04_cpt-6': GeoDataFrame shape: (3937, 1) (2D shapes)
      ├── 'cp_ft06_cpt-3': GeoDataFrame shape: (23044, 1) (2D shapes)
      ├── 'cp_ft06_cpt-6': GeoDataFrame shape: (20768, 1) (2D shapes)
      ├── 'cp_ft08_cpt-1': GeoDataFrame shape: (22901, 1) (2D shapes)
      ├── 'cp_ft08_cpt-8': GeoDataFrame shape: (24987, 1) (2D shapes)
      └── 'sopa_patches': GeoDataFrame shape: (64, 1) (2D

In [108]:
membrane_channels

['Vimentin', 'CD3e', 'panCK', 'CD8', 'COL1A1', 'CD20', 'CD68']

In [107]:
image

<xarray.DataArray 'image' (c: 15, y: 8000, x: 8000)> Size: 2GB
dask.array<from-zarr, shape=(15, 8000, 8000), dtype=uint16, chunksize=(1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates:
  * c        (c) int64 120B 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14
  * y        (y) float64 64kB 0.5 1.5 2.5 3.5 ... 7.998e+03 7.998e+03 8e+03
  * x        (x) float64 64kB 0.5 1.5 2.5 3.5 ... 7.998e+03 7.998e+03 8e+03
Attributes:
    transform:  {'pixels': Identity }

In [112]:
all_channels = list(get_spatial_image(sdata, key='991_subset').c.values)

In [113]:
all_channels

['750_bg',
 '647_bg',
 '555_bg',
 '488_bg',
 'DAPI_bg',
 'Vimentin',
 'CD3e',
 'panCK',
 'CD8',
 'DAPI_1',
 'COL1A1',
 'CD20',
 'CD68',
 'Ki67',
 'DAPI_2']

In [115]:
channels = ['Vimentin', 'CD3e', 'panCK', 'COL1A1', 'CD20']

In [116]:
membrane_channel_index = [all_channels.index(channel) for channel in channels]

In [117]:
membrane_channel_index

[5, 6, 7, 10, 11]

In [118]:
selected_channels_image = image.sel(c=membrane_channel_index)

In [119]:
selected_channels_image

<xarray.DataArray 'image' (c: 5, y: 8000, x: 8000)> Size: 640MB
dask.array<getitem, shape=(5, 8000, 8000), dtype=uint16, chunksize=(1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates:
  * c        (c) int64 40B 5 6 7 10 11
  * y        (y) float64 64kB 0.5 1.5 2.5 3.5 ... 7.998e+03 7.998e+03 8e+03
  * x        (x) float64 64kB 0.5 1.5 2.5 3.5 ... 7.998e+03 7.998e+03 8e+03
Attributes:
    transform:  {'pixels': Identity }

In [121]:
my_channels = ['Vimentin', 'CD3e', 'panCK', 'COL1A1', 'CD20']

In [120]:
sdata = calculate_quantile(sdata, key='991_subset', channels=my_channels, quantile_value=0.75)

Selected channels: [5, 6, 7, 10, 11]


KeyError: "not all values found in index 'c'"

In [122]:
image2 = sdata.images["991_subset"]['scale0'].image

In [123]:
image2

<xarray.DataArray 'image' (c: 15, y: 8000, x: 8000)> Size: 2GB
dask.array<from-zarr, shape=(15, 8000, 8000), dtype=uint16, chunksize=(1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates:
  * c        (c) <U8 480B '750_bg' '647_bg' '555_bg' ... 'CD68' 'Ki67' 'DAPI_2'
  * y        (y) float64 64kB 0.5 1.5 2.5 3.5 ... 7.998e+03 7.998e+03 8e+03
  * x        (x) float64 64kB 0.5 1.5 2.5 3.5 ... 7.998e+03 7.998e+03 8e+03
Attributes:
    transform:  {'pixels': Identity }

In [125]:
sdata = calculate_quantile(sdata, key='991_subset', channels=my_channels, quantile_value=0.75)

In [126]:
sdata

SpatialData object, with associated Zarr store: /Users/jnimoca/Jose_BI/P26_SOPA_seg/20240717_spatialdata_mem.zarr
├── Images
│     ├── '991_subset': DataTree[cyx] (15, 8000, 8000), (15, 4000, 4000), (15, 2000, 2000), (15, 1000, 1000), (15, 500, 500)
│     ├── '991_subset_q75_projection': DataArray[cyx] (1, 8000, 8000)
│     ├── 'max_projection': DataArray[cyx] (1, 8000, 8000)
│     └── 'membrane_q85_projection': DataArray[cyx] (1, 8000, 8000)
└── Shapes
      ├── 'cp_ft02_cpt-1': GeoDataFrame shape: (13542, 1) (2D shapes)
      ├── 'cp_ft02_cpt-8': GeoDataFrame shape: (1, 1) (2D shapes)
      ├── 'cp_ft04_cpt-3': GeoDataFrame shape: (16222, 1) (2D shapes)
      ├── 'cp_ft04_cpt-6': GeoDataFrame shape: (3937, 1) (2D shapes)
      ├── 'cp_ft06_cpt-3': GeoDataFrame shape: (23044, 1) (2D shapes)
      ├── 'cp_ft06_cpt-6': GeoDataFrame shape: (20768, 1) (2D shapes)
      ├── 'cp_ft08_cpt-1': GeoDataFrame shape: (22901, 1) (2D shapes)
      ├── 'cp_ft08_cpt-8': GeoDataFrame shape: (24987, 1)

In [127]:
sdata = calculate_quantile(sdata, key='991_subset', channels=my_channels, quantile_value=0.65)

In [128]:
napari_spatialdata.Interactive(sdata)

2024-07-17 14:32:20.635 | INFO     | napari_spatialdata._view:_on_layer_update:327 - Updating layer.
2024-07-17 14:32:20.636 | INFO     | napari_spatialdata._view:_on_layer_update:327 - Updating layer.
2024-07-17 14:32:22.275 | INFO     | napari_spatialdata._view:_on_layer_update:327 - Updating layer.
2024-07-17 14:32:22.276 | INFO     | napari_spatialdata._view:_on_layer_update:327 - Updating layer.
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/mambaforge/base/envs/SOPA3/lib/python3.10/site-packages/napari/utils/action_manager.py", line 230, in _trigger
    self.trigger(name)
  File "/opt/homebrew/Caskroom/mambaforge/base/envs/SOPA3/lib/python3.10/site-packages/napari/utils/action_manager.py", line 432, in trigger
    return self._actions[name].injected()
  File "/opt/homebrew/Caskroom/mambaforge/base/envs/SOPA3/lib/python3.10/site-packages/in_n_out/_store.py", line 804, in _exec
    result = func(**bound.arguments)
  File "/opt/homebrew/Caskroom/mambaforge/base/en

In [137]:
def quantiles_nuclear_membrane(sdata, key:str, nuclear_channels:list, membrane_channels:list, nuclear_quantile:float=0.9, membrane_quantile:float=0.9):

    nuclear_ArrayData = calculate_quantile(sdata, key, nuclear_channels, nuclear_quantile, return_ArrayData=True)
    nuclear_ArrayData.attrs['channel_name'] = 'Nuclei'
    
    membrane_ArrayData = calculate_quantile(sdata, key, membrane_channels, membrane_quantile, return_ArrayData=True)
    membrane_ArrayData.attrs['channel_name'] = 'Membranes'

    concatenated_array = spatialdata.models.Image2DModel.parse(xarray.concat([nuclear_ArrayData, membrane_ArrayData], dim='c'))

    sdata.images[f'{key}_n{int(nuclear_quantile*100)}_m{int(membrane_quantile*100)}_proj'] = concatenated_array
    return sdata

In [138]:
sdata = quantiles_nuclear_membrane(sdata, key='991_subset', 
                                nuclear_channels=['DAPI_bg', 'DAPI_1'], 
                                membrane_channels=['Vimentin', 'CD3e', 'panCK', 'CD8', 'COL1A1', 'CD20', 'CD68'], 
                                nuclear_quantile=0.9, 
                                membrane_quantile=0.75)

/opt/homebrew/Caskroom/mambaforge/base/envs/SOPA3/lib/python3.10/site-packages/spatialdata/_core/_elements.py:62: UserWarning: Key `991_subset_n90_m75_proj` already exists. Overwriting it in-memory.
  self._check_key(key, self.keys(), self._shared_keys)


In [139]:
sdata

SpatialData object, with associated Zarr store: /Users/jnimoca/Jose_BI/P26_SOPA_seg/20240717_spatialdata_mem.zarr
├── Images
│     ├── '991_subset': DataTree[cyx] (15, 8000, 8000), (15, 4000, 4000), (15, 2000, 2000), (15, 1000, 1000), (15, 500, 500)
│     ├── '991_subset_n90_m75_proj': DataArray[cyx] (2, 8000, 8000)
│     ├── '991_subset_q65_projection': DataArray[cyx] (1, 8000, 8000)
│     ├── '991_subset_q75_projection': DataArray[cyx] (1, 8000, 8000)
│     ├── 'max_projection': DataArray[cyx] (1, 8000, 8000)
│     └── 'membrane_q85_projection': DataArray[cyx] (1, 8000, 8000)
└── Shapes
      ├── 'cp_ft02_cpt-1': GeoDataFrame shape: (13542, 1) (2D shapes)
      ├── 'cp_ft02_cpt-8': GeoDataFrame shape: (1, 1) (2D shapes)
      ├── 'cp_ft04_cpt-3': GeoDataFrame shape: (16222, 1) (2D shapes)
      ├── 'cp_ft04_cpt-6': GeoDataFrame shape: (3937, 1) (2D shapes)
      ├── 'cp_ft06_cpt-3': GeoDataFrame shape: (23044, 1) (2D shapes)
      ├── 'cp_ft06_cpt-6': GeoDataFrame shape: (20768, 1) (

In [142]:
# Define new channel names
new_channel_names = ['Nuclei', 'Membranes']

# Change coordinates of 'c' to new_channel_names
sdata.images['991_subset_n90_m75_proj'] = sdata.images['991_subset_n90_m75_proj'].assign_coords(c=new_channel_names)

/opt/homebrew/Caskroom/mambaforge/base/envs/SOPA3/lib/python3.10/site-packages/spatialdata/_core/_elements.py:62: UserWarning: Key `991_subset_n90_m75_proj` already exists. Overwriting it in-memory.
  self._check_key(key, self.keys(), self._shared_keys)


In [143]:
sdata.images['991_subset_n90_m75_proj']

<xarray.DataArray 'image' (c: 2, y: 8000, x: 8000)> Size: 1GB
dask.array<concatenate, shape=(2, 8000, 8000), dtype=float64, chunksize=(1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates:
    quantile  (c) float64 16B 0.9 0.75
  * y         (y) float64 64kB 0.5 1.5 2.5 3.5 ... 7.998e+03 7.998e+03 8e+03
  * x         (x) float64 64kB 0.5 1.5 2.5 3.5 ... 7.998e+03 7.998e+03 8e+03
  * c         (c) <U9 72B 'Nuclei' 'Membranes'
Attributes:
    transform:     {'pixels': Identity }
    channel_name:  Nuclei

In [144]:
napari_spatialdata.Interactive(sdata)

2024-07-17 15:03:24.122 | INFO     | napari_spatialdata._view:_on_layer_update:327 - Updating layer.
2024-07-17 15:03:24.124 | INFO     | napari_spatialdata._view:_on_layer_update:327 - Updating layer.
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/mambaforge/base/envs/SOPA3/lib/python3.10/site-packages/napari_spatialdata/_widgets.py", line 63, in <lambda>
    self.itemDoubleClicked.connect(lambda item: self._onAction((item.text(),)))
  File "/opt/homebrew/Caskroom/mambaforge/base/envs/SOPA3/lib/python3.10/site-packages/napari_spatialdata/_widgets.py", line 126, in _onAction
    self.viewer.dims.set_point(0, i)
  File "/opt/homebrew/Caskroom/mambaforge/base/envs/SOPA3/lib/python3.10/site-packages/napari/utils/_proxies.py", line 192, in __call__
    return self.create(self.__wrapped__(*args, **kwargs))
  File "/opt/homebrew/Caskroom/mambaforge/base/envs/SOPA3/lib/python3.10/site-packages/napari/components/dims.py", line 252, in set_point
    self.set_current_step(axis

# Quantile calculation for membrane segmentation done

Sopa segmentation with cyto 3

In [146]:
sdata.write('20240717_spatialdata_mem_nuc.zarr')

INFO     The SpatialData object is not self-contained (i.e. it contains some elements that are Dask-backed from    
         locations outside 20240717_spatialdata_mem_nuc.zarr). Please see the documentation of                     
         `is_self_contained()` to understand the implications of working with SpatialData objects that are not     
         self-contained.                                                                                           
INFO     The Zarr backing store has been changed from 20240717_spatialdata_mem.zarr the new file path:             
         20240717_spatialdata_mem_nuc.zarr                                                                         


In [147]:
method = sopa.segmentation.methods.cellpose_patch(diameter=25, channels=["Membranes", "Nuclei"], flow_threshold=0.6, cellprob_threshold=-6, model_type="cyto3")

In [150]:
segmentation = sopa.segmentation.StainingSegmentation(sdata, method, image_key='991_subset_n90_m75_proj', channels=["Membranes", "Nuclei"], min_area=250)

In [161]:
cellpose_temp_dir = "/Users/jnimoca/Jose_BI/P26_SOPA_seg/20240717_spatialdata_mem_nuc.zarr/.sopa_cache/cellpose"

In [163]:
segmentation.write_patches_cells(cellpose_temp_dir)

[WARNING] (sopa.segmentation.stainings) Running segmentation in a sequential manner. This is not recommended on large images because it can be extremely slow (see https://github.com/gustaveroussy/sopa/discussions/36 for more details)
Run all patches:   0%|          | 0/64 [00:00<?, ?it/s]


ValueError: Images of type float must be between -1 and 1.

In [ ]:
sdata

In [178]:
sdata.images['991_subset_n90_m75_proj'].coords

Coordinates:
    quantile  (c) float64 16B 0.9 0.75
  * y         (y) float64 64kB 0.5 1.5 2.5 3.5 ... 7.998e+03 7.998e+03 8e+03
  * x         (x) float64 64kB 0.5 1.5 2.5 3.5 ... 7.998e+03 7.998e+03 8e+03
  * c         (c) <U9 72B 'Nuclei' 'Membranes'

In [168]:
x = sdata.images['991_subset_n90_m75_proj'].values

In [169]:
x.shape

(2, 8000, 8000)

In [170]:
x.max()

14947.1

In [171]:
x.min()

0.0

In [172]:
x.mean()

509.0952747835931

In [173]:
x.dtype

dtype('float64')